In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import jax
import jax.numpy as jnp
from tinyjaxley.modules.comp import Compartment
from tinyjaxley.channels.hh import Na, K, Leak
import matplotlib.pyplot as plt
from tinyjaxley.solvers import fw_euler

In [11]:
comp = Compartment()
comp.record()
comp.add(Leak())
# comp.add(Na())
# comp.add(K())
comp.stimulate(jnp.ones(1000))

for name, channel in comp.channels.items():
    comp.states = channel.init(comp.states, comp.params)

u0 = comp.states
p = comp.params
i_ext = comp.externals["i"]

def step(u, i_ext, dt = 0.025):
    u["i"] = i_ext
    u["v"], _ = fw_euler(comp.dV(u, p, u["t"]), u["v"], u["t"], dt)
    for cname, channel in comp.channels.items():
        for k, du in channel.du({**u, **u[cname]}, {**p, **p[cname]}, u["t"]).items():
            u[cname][k], u[cname]["t"] = fw_euler(du, u[cname][k], u["t"], dt)
    u.pop("i")
    recs = [u[rec] for rec in comp.recordings]
    return u, recs

_, u = jax.lax.scan(step, (u0, 0), i_ext)


TypeError: 'tuple' object does not support item assignment

In [39]:
def test_step(carry, t, dt = 0.025):
    v, _ = carry
    dv = v
    new_v = v + dv * dt
    return (new_v, t + dt), new_v

_, u = jax.lax.scan(test_step, (u0["v"], u0["t"]), jnp.arange(0, 1000, 0.025))


In [40]:
plt.plot(u["v"])

TypeError: JAX does not support string indexing; got idx=('v',)